In [11]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast
import random

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix
import joblib

In [12]:
# Users with Similar Behavior to User 1: 2, 9, 11, 12, ...
# data = {
#     "attraction_id": ["A1", "A10", "A11", "A12", "A13", "A14", "A15", "A17", "A19", "A2", "A20", "A3", "A4", "A5", "A6", "A7", "A8", "A9"],
#     1:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     2:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     3:  [0.0, 4.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#     4:  [5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0],
#     5:  [0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
#     6:  [0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0],
#     7:  [0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0],
#     8:  [0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 0.0, 2.0, 0.0, 4.0, 2.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0],
#     9:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     10: [0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 2.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0],
#     # 11: [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 12: [0.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 13: [5.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0],
#     # 14: [0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0, 1.0, 0.0],
#     # 15: [1.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0],
#     # 16: [0.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 17: [0.0, 1.0, 4.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 18: [4.0, 0.0, 0.0, 5.0, 0.0, 3.0, 2.0, 0.0, 0.0, 3.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0],
#     # 19: [0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0],
#     # 20: [0.0, 5.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 2.0, 0.0, 4.0, 0.0],
# }



user_rating_data = [] # [[user_id_1, attraction_id_1, rating],...]
try:
    API_ENDPOINT = f"http://localhost:3000/api/userrating/getAll"
    res_user_rating = requests.get(url=API_ENDPOINT).json()
    print("check len --> ", len(res_user_rating))
    for cur_res_user_rating in res_user_rating:
        cur_user_id = cur_res_user_rating['userId']

        # users with fewer than 35 ratings have no influence on this model.
        if(len(cur_res_user_rating['rating']) < 35):
            continue
        
        for cur_user_rating in cur_res_user_rating['rating']:
            
            cur_attraction_id = cur_user_rating['attractionId']
            cur_rating_score = cur_user_rating['rating_score']

            user_rating_data.append([cur_user_id, cur_attraction_id, cur_rating_score])


except Exception as e:
    print("get user_rating failed !")
    print(e)

check len -->  478


In [13]:
df = pd.DataFrame(user_rating_data, columns=['user_id', 'attraction_id', 'rating_score'])
df
# Rows = Users, Columns = Attractions
pivot_df = df.pivot_table(
    values = ['rating_score'], 
    index = ['user_id'],
    columns = ['attraction_id'],
    aggfunc='last',
    fill_value=0
)

ratings_matrix = pivot_df
ratings_matrix

rating_score                           \
attraction_id            675c202f9f570448ebeb4827 675c202f9f570448ebeb4829   
user_id                                                                      
677e41b2519209b0aaff4d76                        0                        0   
677e41b6519209b0aaff4d7b                        5                        0   
677e41b9519209b0aaff4d80                        0                        4   
677e41bc519209b0aaff4d85                        0                        0   
677e41bf519209b0aaff4d8a                        0                        0   
...                                           ...                      ...   
6799dce5dd4c73b429f7a8c9                        0                        4   
6799dce9dd4c73b429f7a8ce                        0                        0   
6799dcecdd4c73b429f7a8d3                        4                        0   
6799dcf0dd4c73b429f7a8d8                        4                        0   
6799dcf3dd4c73b429f7a8dd                        0                        5   

                                                                            \
attraction_id            675c202f9f570448ebeb482b 675c202f9f570448ebeb482d   
user_id                                                                      
677e41b2519209b0aaff4d76                        0                        0   
677e41b6519209b0aaff4d7b                        0                        4   
677e41b9519209b0aaff4d80                        3                        5   
677e41bc519209b0aaff4d85                        0                        0   
677e41bf519209b0aaff4d8a                        0                        0   
...                                           ...                      ...   
6799dce5dd4c73b429f7a8c9                        0                        0   
6799dce9dd4c73b429f7a8ce                        5                        0   
6799dcecdd4c73b429f7a8d3                        0                        5   
6799dcf0dd4c73b429f7a8d8                        0                        0   
6799dcf3dd4c73b429f7a8dd                        5                        0   

                                                                            \
attraction_id            675c202f9f570448ebeb482f 675c202f9f570448ebeb4831   
user_id                                                                      
677e41b2519209b0aaff4d76                        0                        0   
677e41b6519209b0aaff4d7b                        0                        4   
677e41b9519209b0aaff4d80                        3                        0   
677e41bc519209b0aaff4d85                        0                        0   
677e41bf519209b0aaff4d8a                        0                        0   
...                                           ...                      ...   
6799dce5dd4c73b429f7a8c9                        0                        0   
6799dce9dd4c73b429f7a8ce                        0                        0   
6799dcecdd4c73b429f7a8d3                        0                        4   
6799dcf0dd4c73b429f7a8d8                        0                        5   
6799dcf3dd4c73b429f7a8dd                        0                        0   

                                                                            \
attraction_id            675c202f9f570448ebeb4833 675c202f9f570448ebeb4835   
user_id                                                                      
677e41b2519209b0aaff4d76                        0                        0   
677e41b6519209b0aaff4d7b                        0                        4   
677e41b9519209b0aaff4d80                        4                        0   
677e41bc519209b0aaff4d85                        0                        0   
677e41bf519209b0aaff4d8a                        0                        0   
...                                           ...                      ...   
6799dce5dd4c73b429f7a8c9                        0       

In [14]:
print(ratings_matrix.values)
print(len(ratings_matrix))

[[0 0 0 ... 5 0 0]
 [5 0 0 ... 5 3 0]
 [0 4 3 ... 3 3 0]
 ...
 [4 0 0 ... 5 5 4]
 [4 0 0 ... 4 4 5]
 [0 5 5 ... 5 5 0]]
337


In [15]:
''' 
example user id of following persona (that have rating):
- user_persona_nature --> 677e41b2519209b0aaff4d76, 677e41b6519209b0aaff4d7b
- user_persona_adventure --> 677e4327519209b0aaff4e3e, 677e432a519209b0aaff4e43
- user_persona_art --> 677e544b519209b0aaff4fd6, 677e544e519209b0aaff4fdb
- user_persona_temple_historical --> 677e425a519209b0aaff4dda, 677e425d519209b0aaff4ddf
'''

# use '677e4327519209b0aaff4e3e' ('love_adventure_0') for better example
# as this 'love_adventure_0' have similar users rating art attractions that 'love_adventure_0'  nerver been there and vote before

user_id = '677e41b2519209b0aaff4d76'

knn = NearestNeighbors(metric='cosine')
knn.fit(ratings_matrix)

# Find the user's ratings
row_position = ratings_matrix.index.get_loc(user_id)
# Get 10 neighbors(10 similar user)
user_rating_vector = ratings_matrix.iloc[row_position, :].values.reshape(1,-1)
distances, indices = knn.kneighbors(user_rating_vector, n_neighbors=10+1)

print(indices)

# Get similar users(exclude the target user itself)
similar_users = indices.flatten()
similar_users = similar_users[similar_users != row_position]
print("check similar_users", similar_users)

# Aggregate attraction ratings from similar users
similar_users_ratings = ratings_matrix.iloc[similar_users].mean(axis=0)

# Exclude already rated attractions by the user
user_ratings = ratings_matrix.iloc[row_position]
recommendations = similar_users_ratings[user_ratings == 0]

# Sort recommendations by highest score
recommendations = recommendations.sort_values(ascending=False)
# save model
joblib.dump(knn, 'model_collaborative_filtering.joblib')


print(recommendations)
print("Idx of reccommed")
res_reccomend = list(recommendations.reset_index()['attraction_id'].values)
print(res_reccomend[:min(30, len(res_reccomend))])

[[  0 223 324 126 329 333 131 118 125 326 234]]
check similar_users [223 324 126 329 333 131 118 125 326 234]
              attraction_id           
rating_score  675c20349f570448ebeb4968    4.8
              675c20359f570448ebeb4a16    4.7
              675c20339f570448ebeb4946    4.6
              675c20329f570448ebeb48f8    4.6
              675c20359f570448ebeb49c8    4.6
                                         ... 
              675c20379f570448ebeb4ac6    0.0
              675c20379f570448ebeb4ad8    0.0
              675c20379f570448ebeb4ace    0.0
              675c20389f570448ebeb4af6    0.0
              675c20389f570448ebeb4b16    0.0
Length: 166, dtype: float64
Idx of reccommed
['675c20349f570448ebeb4968', '675c20359f570448ebeb4a16', '675c20339f570448ebeb4946']


In [16]:
print(recommendations.reset_index()['attraction_id'].values)
print(type(recommendations.reset_index()['attraction_id'].values))
print(list(recommendations.reset_index()['attraction_id'].values))

['675c20349f570448ebeb4968' '675c20359f570448ebeb4a16'
 '675c20339f570448ebeb4946' '675c20329f570448ebeb48f8'
 '675c20359f570448ebeb49c8' '675c20349f570448ebeb497c'
 '675c20359f570448ebeb4a0c' '675c20319f570448ebeb48f0'
 '675c20369f570448ebeb4a84' '675c20339f570448ebeb494e'
 '675c20319f570448ebeb48f2' '675c20349f570448ebeb49ba'
 '675c20359f570448ebeb4a20' '675c20339f570448ebeb494c'
 '675c20369f570448ebeb4a78' '675c20369f570448ebeb4a82'
 '675c20389f570448ebeb4b10' '675c20359f570448ebeb49cc'
 '675c20369f570448ebeb4a88' '675c20369f570448ebeb4a26'
 '675c20359f570448ebeb49da' '675c20379f570448ebeb4a98'
 '675c20339f570448ebeb4932' '675c20309f570448ebeb4885'
 '675c202f9f570448ebeb4863' '675c20329f570448ebeb4902'
 '675c20359f570448ebeb4a08' '675c20349f570448ebeb4980'
 '675c20389f570448ebeb4b0c' '675c20309f570448ebeb4873'
 '675c20319f570448ebeb48d4' '675c20359f570448ebeb4a12'
 '675c20309f570448ebeb4891' '675c202f9f570448ebeb4867'
 '675c20389f570448ebeb4aec' '675c202f9f570448ebeb482b'
 '675c2030

In [17]:
recommendations

              attraction_id           
rating_score  675c20349f570448ebeb4968    4.8
              675c20359f570448ebeb4a16    4.7
              675c20339f570448ebeb4946    4.6
              675c20329f570448ebeb48f8    4.6
              675c20359f570448ebeb49c8    4.6
                                         ... 
              675c20379f570448ebeb4ac6    0.0
              675c20379f570448ebeb4ad8    0.0
              675c20379f570448ebeb4ace    0.0
              675c20389f570448ebeb4af6    0.0
              675c20389f570448ebeb4b16    0.0
Length: 166, dtype: float64

In [18]:
recommendations.reset_index()

,level_0,attraction_id,0
0,rating_score,675c20349f570448ebeb4968,4.8
1,rating_score,675c20359f570448ebeb4a16,4.7
2,rating_score,675c20339f570448ebeb4946,4.6
3,rating_score,675c20329f570448ebeb48f8,4.6
4,rating_score,675c20359f570448ebeb49c8,4.6
...,...,...,...
161,rating_score,675c20379f570448ebeb4ac6,0.0
162,rating_score,675c20379f570448ebeb4ad8,0.0
163,rating_score,675c20379f570448ebeb4ace,0.0
164,rating_score,675c20389f570448ebeb4af6,0.0


In [19]:
{
    "preference_prompt" : [
        {"name": "Nature & Outdoors", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy spending time in nature, such as mountains, beaches, national parks, waterfalls, forests, or riversides?"},

        {"name": "Cultural & Historical", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy exploring cultural and historical attractions like temples, museums, landmarks, or monuments?"},

        {"name": "Urban & City Life", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy urban experiences like shopping, walking streets, cityscapes, markets, or nightlife?"},

        {"name": "Relaxation & Meditation", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy relaxing activities like meditation, cozy snuggles, or peaceful foggy settings?"},

        {"name": "Adventure & Activities", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy adventure activities like hiking, cycling, camping, or diving?"},

        {"name": "Wildlife & Family-Friendly", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy visiting family-friendly places like zoos or waterparks with activities suitable for kids?"},

        {"name": "Arts & Architecture", "score": 1, 
        "question": "On a scale of 0-5, how much do you enjoy exploring art and architecture during your travels?"},

        {"name": "Small Villages & Rural Life", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy visiting small villages and experiencing rural life?"},

        {"name": "Islands & Beaches", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy visiting islands and spending time on beaches?"}
    ]
}

{'preference_prompt': [{'name': 'Nature & Outdoors',
   'score': 0,
   'question': 'On a scale of 0-5, how much do you enjoy spending time in nature, such as mountains, beaches, national parks, waterfalls, forests, or riversides?'},
  {'name': 'Cultural & Historical',
   'score': 0,
   'question': 'On a scale of 0-5, how much do you enjoy exploring cultural and historical attractions like temples, museums, landmarks, or monuments?'},
  {'name': 'Urban & City Life',
   'score': 0,
   'question': 'On a scale of 0-5, how much do you enjoy urban experiences like shopping, walking streets, cityscapes, markets, or nightlife?'},
  {'name': 'Relaxation & Meditation',
   'score': 0,
   'question': 'On a scale of 0-5, how much do you enjoy relaxing activities like meditation, cozy snuggles, or peaceful foggy settings?'},
  {'name': 'Adventure & Activities',
   'score': 0,
   'question': 'On a scale of 0-5, how much do you enjoy adventure activities like hiking, cycling, camping, or diving?'},
  

In [20]:
{"b":[ {"name": "Islands & Beaches", "score": 0, 
        "question": "On a scale of 0-5, how much do you enjoy visiting islands and spending time on beaches?"}
]}

{'b': [{'name': 'Islands & Beaches',
   'score': 0,
   'question': 'On a scale of 0-5, how much do you enjoy visiting islands and spending time on beaches?'}]}

In [21]:
print(sys.path)

['c:\\Program Files\\Python313\\python313.zip', 'c:\\Program Files\\Python313\\DLLs', 'c:\\Program Files\\Python313\\Lib', 'c:\\Program Files\\Python313', '', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python313\\site-packages', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python313\\site-packages\\win32', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python313\\site-packages\\win32\\lib', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python313\\site-packages\\Pythonwin', 'c:\\Program Files\\Python313\\Lib\\site-packages', '..', '..']


: 